In [1]:
using Flux
using Flux.Optimise: update!
using Flux.Losses: mae, mse
using DiffEqFlux
using DiffEqSensitivity
using DifferentialEquations

using ProgressBars
using LinearAlgebra

using Random
using ProgressBars
using DelimitedFiles
using Plots, Colors, Printf, Profile

┌ Warning: Error requiring `EnzymeCore` from `KernelAbstractions`
│   exception = (LoadError("/home/zhuxu21/.julia/packages/KernelAbstractions/zPAn3/ext/EnzymeExt.jl", 1, UndefVarError(:EnzymeRules)), Union{Ptr{Nothing}, Base.InterpreterIP}[Ptr{Nothing} @0x00007f9ccd22b91f, Ptr{Nothing} @0x00007f9ccd260af6, Ptr{Nothing} @0x00007f9ccd2820ec, Ptr{Nothing} @0x00007f9ccd2604da, Ptr{Nothing} @0x00007f9ccd261171, Base.InterpreterIP in top-level CodeInfo for KernelAbstractions.EnzymeExt at statement 6, Ptr{Nothing} @0x00007f9ccd28100a, Ptr{Nothing} @0x00007f9ccd281847, Ptr{Nothing} @0x00007f9ccd281a28, Ptr{Nothing} @0x00007f9ccd282e6a, Ptr{Nothing} @0x00007f9cb9b94281, Ptr{Nothing} @0x00007f9ccd242b3d, Ptr{Nothing} @0x00007f9cb9b056c1, Ptr{Nothing} @0x00007f9cb93ea930, Ptr{Nothing} @0x00007f9cb93ea94f, Ptr{Nothing} @0x00007f9ccd242b3d, Ptr{Nothing} @0x00007f9ccd2519f8, Ptr{Nothing} @0x00007f9cb75c937b, Ptr{Nothing} @0x00007f9cb75c93a2, Ptr{Nothing} @0x00007f9ccd242b3d, Ptr{Nothing} @0x00007f9

In [2]:
# using Distributed
# t = @async addprocs(8)
BLAS.set_num_threads(1)
@show Threads.nthreads();

Threads.nthreads() = 1


In [3]:
# Problem settings
function robertson(dy, y, p, t)
    k = [0.04, 3e7, 1e4] .* p
    dy[1] = -k[1]*y[1]+k[3]*y[2]*y[3]
    dy[2] =  k[1]*y[1]-k[2]*y[2]^2-k[3]*y[2]*y[3]
    dy[3] =  k[2]*y[2]^2
end

y0 = [1.0, 0.0, 0.0]
datasize = 100 # 100
tsteps = 10 .^ range(log10(1e-4), log10(1e8), length=datasize)
tspan = (0.0, tsteps[end]+1e-3)

p_true = [1, 1, 1]
weights = [1, 2e4, 1]

# solver = DifferentialEquations.CVODE_BDF()
solver = KenCarp4();
prob = ODEProblem(robertson, y0, tspan, p_true);
y_true = solve(prob, solver, p=p_true, saveat=tsteps);

In [4]:
# define the problem
scale = maximum(y_true, dims=2);
function predict_ode(p) # Our 1-layer "neural network"
    sol = solve(prob, solver, p=p, saveat=tsteps)
    pred = Array(sol)
    if _sol.retcode == ReturnCode.Success
        return _pred
    else
        println("ode solver failed")
    end
end
function loss_ode(p; y_train=y_true) 
    return sum(abs2, (predict_ode(p) - y_train) ./ scale);
end

# settings for plots
Plots.pyplot()
Plots.resetfontsizes()
Plots.scalefontsizes(1.5)
function valid(tsteps, y_true, y_pred; title="", line1=(3, :scatter), line2=(2, :solid))
    IJulia.clear_output(true)
    h = plot(legend=false, title=title, size=(600,350), framestyle=:box, 
             palette=palette(:darktest, length(weights)))
    plot!(tsteps, (y_true .* weights)', xscale=:log10, line=line1, msw=0.0)
    plot!(tsteps, (y_pred .* weights)', xscale=:log10, line=line2)
    return h
end
# valid(tsteps, y_true, y_pred, title="test")

valid (generic function with 1 method)

train (generic function with 1 method)

In [6]:
rng = MersenneTwister(0x7777777);
n_exp = 100;
n_epoch = 300;
p_init_arr = exp.(rand(rng, length(p_true), n_exp) .* 2 .- 1);
noise_level_arr = 10 .^ (rand(rng, n_exp) .* (log10(0.2) - log10(10^-3)) .+ log10(10^-3)); # [(0.1% ~ 20%)]

losses_y_exps = zeros(n_exp, n_epoch);
history_p_exps = zeros(n_exp, n_epoch, length(p_true));

In [7]:
function train(i_exp, p_init, noise_level; n_epoch=300, opt = ADAM(0.1))
    rng = MersenneTwister(0x7777777 * i_exp);
    p_pred = deepcopy(p_init);
    y_pred = predict_ode(p_pred)
    y_noise = y_true + noise_level .* (rand(rng, length(y0), length(tsteps)).-0.5) .* scale;
    
    losses_y = Vector{Float64}()
    history_p = Vector{Array{Float64}}();
    for epoch in 1:n_epoch
        grad = Flux.gradient(x -> loss_ode(x; y_train=y_noise), p_pred)[1]
        update!(opt, p_pred, grad)
        
        loss_y = loss_ode(p_pred; y_train=y_noise)
        push!(losses_y, deepcopy(loss_y))
        push!(history_p, deepcopy(p_pred))
    end
    return losses_y, history_p;
end

for i in 1:100
    p_init = p_init_arr[:,i];
    noise_level = noise_level_arr[i];
    
    losses_y, history_p = train(i, p_init, noise_level; n_epoch=300);
    losses_y_exps[i,:] = deepcopy(losses_y);
    history_p_exps[i,:,:] = deepcopy(hcat(history_p...)');
    
    @show i, losses_y[end], history_p[end];
end

ode solver failed
ode solver failed


FunctionWrappersWrappers.NoFunctionWrapperFoundError: No matching function wrapper was found!

In [8]:
h = plot(legend=false)
for i in 1:10
    plot!(tsteps, (y_init .* weights)', xscale=:log10, color = "#BBBBBB", alpha=0.3)
    # plot!(tsteps, (y_pred .* weights)', xscale=:log10, color = "#BB0000", alpha=0.3)
end
plot!(tsteps, (y_true .* weights)', xscale=:log10)
display(h)

UndefVarError: UndefVarError: y_init not defined

In [9]:
n_solved = 100;
losses_p_exps = zeros(n_exp, n_epoch);
errors_p_exps = zeros(n_exp, n_epoch);
for i=1:n_solved
    losses_p_exps[i,:] = sum(abs2.(history_p_exps[i,:,:] .- 1), dims=2)
    errors_p_exps[i,:] = sum(abs.(history_p_exps[i,:,:] .- 1), dims=2)./3
end

h1 = plot(xlabel="Noise", ylabel="Final y loss", size=(400,300), legend=false);
plot!(noise_level_arr[1:n_solved], (losses_y_exps[1:n_solved, end] ./ datasize ./ 3) .^ 0.5,  xscale=:log10, yscale=:log10, line=(3, :scatter), color=:black)

h2 = plot(xlabel="Noise", ylabel="Final p loss", size=(400,300), legend=false);
plot!(noise_level_arr[1:n_solved], (losses_p_exps[1:n_solved, end] ./ 3) .^ 0.5,  xscale=:log10, yscale=:log10, line=(3, :scatter), color=:black)

h = plot(h1, h2, layout=(2,1), size=(400,400), framestyle=:box)
Plots.savefig(h, "figures/Robertson_final_losses.svg")

SystemError: SystemError: opening file "/data/ZhuXu/Cantera/ArrheniusOpt/example/Robertson/ipynb/figures/Robertson_final_losses.svg": No such file or directory

In [10]:
h1 = plot(xlabel="Epochs", ylabel="y loss", size=(400,300), legend=false);
for i=1:n_solved
    a = sqrt(noise_level_arr[i] / 10^-0.5)
    plot!(1:n_epoch, (losses_y_exps[i, :] ./ datasize ./ 3) .^ 0.5, yscale=:log10, line=(1, :solid), alpha=a, color=:black)
end

h2 = plot(xlabel="Epochs", ylabel="p loss", size=(400,300), legend=false);
for i=1:n_solved
    a = sqrt(noise_level_arr[i] / 10^-0.5)
    plot!(1:n_epoch, (losses_p_exps[i, :] ./ 3) .^ 0.5, yscale=:log10, line=(1, :solid), alpha=a, color=:black)
end
h = plot(h1, h2, layout=(2,1), size=(400,400), framestyle=:box)
Plots.savefig(h, "figures/Robertson_losses.svg")

SystemError: SystemError: opening file "/data/ZhuXu/Cantera/ArrheniusOpt/example/Robertson/ipynb/figures/Robertson_losses.svg": No such file or directory

In [11]:
# i=1;
# p_pred = p_init_arr[:,i];
# ps = Flux.params(p_pred)
# function loss()
#     return sum(abs2, (predict_ode(p_pred) - y_true) ./ scale);
# end
# opt = ADAM(0.1)
# data = Iterators.repeated((), 300);
# @time Flux.train!(loss, ps, data, opt);
# # about 67 secs; manualy 80 secs

In [12]:
# single case 1: without noise
i = 100;
p_init = p_init_arr[:,i];
noise_level = 0.;

n0_losses_y, n0_history_p = train(i, p_init, noise_level; n_epoch=300);

ode solver failed
ode solver failed


FunctionWrappersWrappers.NoFunctionWrapperFoundError: No matching function wrapper was found!

In [13]:
i = 100;
p_init = p_init_arr[:,i];
noise_level = 0.;

rng = MersenneTwister(0x7777777 * i);
y_noise = y_true + noise_level .* (rand(rng, length(y0), length(tsteps)).-0.5) .* scale;
p_pred = n0_history_p[end]

y_init = predict_ode(p_init)
h = valid(tsteps, y_noise, y_init; title="$(@sprintf("p=[%.3f  %.3f  %.3f]", p_init[1], p_init[2], p_init[3]))")
Plots.savefig(h, "figures/Robertson_no_noise_init.svg")

UndefVarError: UndefVarError: n0_history_p not defined

In [14]:
y_pred = predict_ode(p_pred)
valid(tsteps, y_noise, y_pred)
h = valid(tsteps, y_noise, y_pred; title="$(@sprintf("p=[%.3f  %.3f  %.3f]", p_pred[1], p_pred[2], p_pred[3]))")
Plots.savefig(h, "figures/Robertson_no_noise_pred.svg")

UndefVarError: UndefVarError: p_pred not defined

In [15]:
# single case 2: 30% noise
i=100
p_init = p_init_arr[:,i];
noise_level = 0.1;

n3_losses_y, n3_history_p = train(i, p_init, noise_level; n_epoch=300);

ode solver failed
ode solver failed


FunctionWrappersWrappers.NoFunctionWrapperFoundError: No matching function wrapper was found!

In [16]:
i=100
p_init = p_init_arr[:,i];
noise_level = 0.2;

rng = MersenneTwister(0x7777777 * i);
y_noise = y_true + noise_level .* (rand(rng, length(y0), length(tsteps)).-0.5) .* scale;
p_pred = n3_history_p[end]

y_init = predict_ode(p_init)
h = valid(tsteps, y_noise, y_init; title="$(@sprintf("p=[%.3f  %.3f  %.3f]", p_init[1], p_init[2], p_init[3]))")
Plots.savefig(h, "figures/Robertson_0.2_noise_init.svg")

UndefVarError: UndefVarError: n3_history_p not defined

In [17]:
y_pred = predict_ode(p_pred)
valid(tsteps, y_noise, y_pred)
h = valid(tsteps, y_noise, y_pred; title="$(@sprintf("p=[%.3f  %.3f  %.3f]", p_pred[1], p_pred[2], p_pred[3]))")
Plots.savefig(h, "figures/Robertson_0.2_noise_pred.svg")

UndefVarError: UndefVarError: p_pred not defined

In [18]:
@show sqrt(sum((n0_history_p[end] .- 1).^2 / 3));
@show sqrt(sum((n3_history_p[end] .- 1).^2 / 3));

UndefVarError: UndefVarError: n0_history_p not defined